# Pine Script ML Indicator Optimizer - Google Colab

This notebook clones the repository, installs dependencies, and launches the interactive optimizer.

## Instructions
1. Run the cells in order
2. Use the interactive menu to download historical data and configure optimization parameters

## Step 1: Clone Repository and Setup

In [ ]:
import os
import sys
import subprocess
from pathlib import Path

# Clone the repository
repo_url = "https://github.com/yanniedog/pinescript-ml-optimiser.git"
repo_name = "pinescript-ml-optimiser"

# Remove existing directory if it exists (for re-running)
if os.path.exists(repo_name):
    print(f"Removing existing {repo_name} directory...")
    import shutil
    shutil.rmtree(repo_name)

print(f"Cloning repository from {repo_url}...")
result = subprocess.run(["git", "clone", repo_url], capture_output=True, text=True)
if result.returncode != 0:
    print(f"Error cloning repository: {result.stderr}")
    raise RuntimeError("Failed to clone repository")
print(result.stdout)

# Change to the repository directory
os.chdir(repo_name)
print(f"Changed working directory to: {os.getcwd()}")

# Add the repository directory to Python path
repo_path = Path(os.getcwd()).resolve()
if str(repo_path) not in sys.path:
    sys.path.insert(0, str(repo_path))
    print(f"Added {repo_path} to Python path")

print("\n[OK] Repository cloned and setup complete!")

## Step 2: Install Dependencies

In [ ]:
# Install requirements
print("Installing dependencies from requirements.txt...")
print("-" * 50)

!pip install -q -r requirements.txt

print("\n[OK] Dependencies installed successfully!")

## Step 2.5: Setup Ngrok Tunnel

In [ ]:
# Setup ngrok to expose Plotly server
print("Setting up ngrok tunnel...")
print("-" * 50)

# Install pyngrok if not already installed
try:
    import pyngrok
except ImportError:
    print("Installing pyngrok...")
    !pip install -q pyngrok>=5.0.0
    import pyngrok

# Configure ngrok authtoken
ngrok_authtoken = "1tve5X0Aea2r2LKv82JymsjmSD9_4Xp7L4YWxE7RuCSU9bdHZ"
pyngrok.set_auth_token(ngrok_authtoken)

# Start ngrok tunnel to port 9107 (where Plotly server will run)
print("Starting ngrok tunnel to port 9107...")
tunnel = pyngrok.connect(9107, "http")

# Get the public URL
public_url = tunnel.public_url
print(f"\n[OK] Ngrok tunnel established!")
print(f"Public URL: {public_url}")
print(f"\nThe Plotly dashboard will be accessible at: {public_url}")
print("Note: Keep this notebook running to maintain the tunnel connection.")

## Step 3: Launch Interactive Optimizer

In [ ]:
# Import and run the interactive optimizer
import sys
from screen_log import enable_screen_log

# Enable screen logging (creates logs directory if needed)
enable_screen_log()

# Import the interactive optimizer
try:
    from interactive_optimizer import main as run_optimizer
    
    print("\n" + "="*70)
    print("Starting Interactive Optimizer...")
    print("="*70 + "\n")
    
    # Run the optimizer - handle SystemExit gracefully for Jupyter
    try:
        run_optimizer()
    except SystemExit as e:
        # sys.exit() raises SystemExit - catch it so notebook doesn't crash
        if e.code == 0:
            print("\n[INFO] Optimizer exited successfully.")
        else:
            print(f"\n[INFO] Optimizer exited with code {e.code}.")
    except KeyboardInterrupt:
        print("\n\n[INFO] Interrupted by user.")
    except Exception as e:
        print(f"\n[ERROR] An error occurred: {e}")
        import traceback
        traceback.print_exc()
        
except ImportError as e:
    print(f"[ERROR] Failed to import optimizer: {e}")
    print("\nPlease ensure:")
    print("  1. The repository was cloned successfully")
    print("  2. Dependencies were installed")
    print("  3. You're in the correct directory")
    import traceback
    traceback.print_exc()